## OpenAI Function Calling

In [22]:
import openai
import json

def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return { "event_id": "1234", "status": "Meeting scheduled successfully!",
            "date": date, "time": time, "attendees": attendees }

OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [23]:
# Our predefined function JSON schema:
functions = [
    {
        "name": "schedule_meeting",
        "description": "Set a meeting at a specified date and time for designated attendees",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {"type": "string", "format": "date"},
                "time": {"type": "string", "format": "time"},
                "attendees": {"type": "array", "items": {"type": "string"}}
            },
            "required": ["date", "time", "attendees"]
        }
    }
]

In [24]:
# Start the conversation:
messages = [{"role": "user", "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob."}]

# Send the conversation and function schema to the model:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto"
)

response_message = response["choices"][0]["message"]

# Check if the model wants to call our function:
if response_message.get("function_call"):
    # Extract the arguments:
    function_args = json.loads(response_message["function_call"]["arguments"])
    print("These are the function arguments: ", function_args)

    # Find the function to call:
    function_name = response_message["function_call"]["name"]
    print("This is the function name: ", function_name)
    function = OPENAI_FUNCTIONS.get(function_name)

    if not function:
        raise Exception(f"Function {function_name} not found.")

    # Call the function:
    function_response = function(**function_args)

    # Share the function's response with the model:
    messages.append({
        "role": "function",
        "name": "schedule_meeting",
        "content": json.dumps(function_response)
    })

    # Let the model generate a user-friendly response:
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages
    )

    print(second_response["choices"][0]["message"]["content"])

These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
Meeting scheduled successfully! 
- Date: 2023-11-01 
- Time: 14:00 
- Attendees: Alice, Bob 
- Event ID: 1234


---

## Extracting Data with LangChain

In [28]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field

class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(None, description="Any contrarian points acknowledged in the article")
    author: Optional[str] = Field(None, description="Author of the article")

# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a proficient algorithm capable of extracting structured information."),
        ("human", "*Extract* key points and any contrarian positions from the provided input using the specified format: {input}"),
        ("human", "*Remember to* produce your answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Article, llm, prompt, verbose=True)
chain.run("In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a proficient algorithm capable of extracting structured information.
Human: *Extract* key points and any contrarian positions from the provided input using the specified format: In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.
Human: *Remember to* produce your answer in the correct format

> Finished chain.


Article(points='The growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.', author='Dr. Jane Smith')